In [1]:
# Load libraries 
import os, gc, pickle, scipy.sparse
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from colorama import Fore, Back, Style
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import PercentFormatter

from sklearn.model_selection import KFold
# from sklearn.model_selection import GroupKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error

# Load data
DATA_DIR = "/kaggle/input/open-problems-multimodal/"
FP_CELL_METADATA = os.path.join(DATA_DIR,"metadata.csv")

FP_CITE_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_cite_inputs.h5")
FP_CITE_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_cite_targets.h5")
FP_CITE_TEST_INPUTS = os.path.join(DATA_DIR,"test_cite_inputs.h5")

FP_MULTIOME_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_multi_inputs.h5")
FP_MULTIOME_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_multi_targets.h5")
FP_MULTIOME_TEST_INPUTS = os.path.join(DATA_DIR,"test_multi_inputs.h5")

FP_SUBMISSION = os.path.join(DATA_DIR,"sample_submission.csv")
FP_EVALUATION_IDS = os.path.join(DATA_DIR,"evaluation_ids.csv")

# Set params for model run
LOADFILTER = True
LOADSVD = True
CROSS_VALIDATE = True
SUBMIT = True

In [2]:
def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)

# Load data

In [3]:
if not LOADFILTER:
    %%time
    # load training data
    X_train = pd.read_hdf(FP_CITE_TRAIN_INPUTS)
    print(f"X_train shape: {str(X_train.shape):14} {X_train.size*4/1024/1024/1024:2.3f} GByte")
    # load test data
    X_test = pd.read_hdf(FP_CITE_TEST_INPUTS)
    print(f"X_test shape: {str(X_test.shape):14} {X_test.size*4/1024/1024/1024:2.3f} GByte")
    # load test targets
    y_train = pd.read_hdf(FP_CITE_TRAIN_TARGETS)

    y_columns = list(y_train.columns)
    # y_rows = list(y_train.index)
    # y_train = y_train.values

    print(f"y_train shape: {str(y_train.shape):14} {y_train.size*4/1024/1024/1024:2.3f} GByte")


In [4]:
# metadata
metadata_df = pd.read_csv(FP_CELL_METADATA, index_col='cell_id')
# citeseq
metadata_df_citeseq = metadata_df[metadata_df.technology=="citeseq"]
print(metadata_df_citeseq.shape)
# # multiome
# metadata_df2 = metadata_df[metadata_df.technology=="multiome"]
# print(metadata_df2.shape)

(119651, 4)


## Filter training and test for important features
Inspiration taken from https://www.kaggle.com/code/ambrosm/msci-citeseq-quickstart/

Starter templates have defined two sets of features:
- `constant_cols` is the set of all features which are constant in the train training. These columns will be discarded immediately after loading.
- `important_cols` is the set of all features whose name matches the name of a target protein. If a gene is named 'ENSG00000114013_CD86', it should be related to a protein named 'CD86'. These features will be used for the model unchanged, that is, they don't undergo dimensionality reduction. 

It is likely not all important columns will be represented with this filtering as some genes won't have the exact protein label.
Also, it seems like there's no concern for including important columns as part of SVD.

In [5]:
if not LOADFILTER:
    # get columns where all are zero
    constant_cols = list(X_train.columns[(X_train == 0).all(axis=0).values]) + list(X_test.columns[(X_test == 0).all(axis=0).values])

    important_cols = []
    for y_col in y_train.columns:
        important_cols += [x_col for x_col in X_train.columns if y_col in x_col and x_col not in constant_cols]

    #save pickle files    
    with open('constant_cols.pkl', 'wb') as f:
        pickle.dump(constant_cols, f)
    with open('important_cols.pkl', 'wb') as f:
        pickle.dump(important_cols, f)


In [6]:
if LOADFILTER:
    # load pickle files 
    with open('/kaggle/input/citeseqtruncatedsvd512/constant_cols.pkl', 'rb') as f:
        constant_cols = pickle.load(f)
    with open('/kaggle/input/citeseqtruncatedsvd512/important_cols.pkl', 'rb') as f:
        important_cols = pickle.load(f)
    # set y_train to just values for training
    #y_train = y_train.values


In [7]:
%%time
# Filter out constant genes and training set to sparse matrix
X_train = pd.read_hdf(FP_CITE_TRAIN_INPUTS).drop(columns=constant_cols)
cell_index = X_train.index
meta = metadata_df_citeseq.reindex(cell_index)
X0 = X_train[important_cols].values
# X_train = X_train.drop(columns = important_cols) # not sure whether to exclude important cols from SVD as they will be added into training set after SVD
print(f"X_train shape after filtering: {str(X_train.shape):14} {X_train.size*4/1024/1024/1024:2.3f} GByte")
gc.collect()
X_train = scipy.sparse.csr_matrix(X_train.values)
gc.collect()


# Filter out constant genes and test set to sparse matrix
X_test = pd.read_hdf(FP_CITE_TEST_INPUTS).drop(columns=constant_cols)
cell_index_test = X_test.index
meta_test = metadata_df_citeseq.reindex(cell_index_test)
X0t = X_test[important_cols].values
# X_test = X_test.drop(columns = important_cols)
print(f"X_test shape after filtering: {str(X_test.shape):14} {X_test.size*4/1024/1024/1024:2.3f} GByte")
gc.collect()
X_test = scipy.sparse.csr_matrix(X_test.values)
gc.collect()

X_train shape after filtering: (70988, 20856) 5.515 GByte
X_test shape after filtering: (48663, 20856) 3.781 GByte
CPU times: user 2min 31s, sys: 22.7 s, total: 2min 54s
Wall time: 3min 23s


0

# Apply Truncated SVD

In [8]:
%%time
if not LOADSVD:
    # Apply the singular value decomposition 
    both = scipy.sparse.vstack([X_train, X_test])
    assert both.shape[0] == 119651
    print(f"Shape of both before SVD: {both.shape}")
    svd = TruncatedSVD(n_components=512, random_state=1) # 512
    both = svd.fit_transform(both)
    print(f"Shape of both after SVD:  {both.shape}")

    # save truncated training and test sets
    X_train = both[:70988]
    X_test = both[70988:]
    with open('train_Citeseq_truncated_512.pkl', 'wb') as f:
        pickle.dump(X_train, f)

    with open('test_Citeseq_truncated_512.pkl', 'wb') as f:
        pickle.dump(X_test, f)


In [9]:
if LOADSVD:
    with open('/kaggle/input/citeseqtruncatedsvd512/train_Citeseq_truncated_512.pkl','rb') as f: X_train = pickle.load(f)
    with open('/kaggle/input/citeseqtruncatedsvd512/test_Citeseq_truncated_512.pkl','rb') as f: X_test = pickle.load(f)
    X_train.shape, X_test.shape

# Hstack the svd training set (basically everything that isn't constant) with the important features
X_train = np.hstack([X_train, X0])
X_test = np.hstack([X_test, X0t])
print(f"Reduced X shape:  {str(X_train.shape):14} {X_train.size*4/1024/1024/1024:2.3f} GByte")
print(f"Reduced Xt shape: {str(X_test.shape):14} {X_test.size*4/1024/1024/1024:2.3f} GByte")

Reduced X shape:  (70988, 656)   0.173 GByte
Reduced Xt shape: (48663, 656)   0.119 GByte


In [10]:
# load training targets
# use values only for training
y_train = pd.read_hdf(FP_CITE_TRAIN_TARGETS)
y_train = y_train.values

# Model

## linear regression with Ridge regularization

In [11]:
from sklearn.linear_model import Ridge
def create_model():
    model = Ridge(copy_X=False)
    return model

# Cross validation

5 fold cross validation to QC model.
This should flag problems like overfitting or selection bias if results from cross validation are poor.

In [12]:
%%time
# 5-fold Cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
score_list = []
for fold, (idx_tr, idx_va) in enumerate(kf.split(X_train)):
    model = None
    gc.collect()
    X_tr = X_train[idx_tr] # creates a copy, https://numpy.org/doc/stable/user/basics.copies.html
    y_tr = y_train[idx_tr]
    del idx_tr
    
    model = create_model()
    model.fit(X_tr, y_tr)
    del X_tr, y_tr
    gc.collect()

    # We validate the model
    X_va = X_train[idx_va]
    y_va = y_train[idx_va]
    del idx_va
    y_va_pred = model.predict(X_va)
    mse = mean_squared_error(y_va, y_va_pred)
    corrscore = correlation_score(y_va, y_va_pred)
    del X_va, y_va

    print(f"Fold {fold}: mse = {mse:.5f}, corr =  {corrscore:.3f}")
    score_list.append((mse, corrscore))

# Show overall score
result_df = pd.DataFrame(score_list, columns=['mse', 'corrscore'])
print(f"{Fore.GREEN}{Style.BRIGHT}{X_train.shape} Average  mse = {result_df.mse.mean():.5f}; corr = {result_df.corrscore.mean():.3f}{Style.RESET_ALL}")
result_df.to_csv('citeseq_ridge_crossval_res.csv')

Fold 0: mse = 2.54443, corr =  0.893
Fold 1: mse = 2.54259, corr =  0.893
Fold 2: mse = 2.54726, corr =  0.893
Fold 3: mse = 2.55948, corr =  0.892
Fold 4: mse = 2.54676, corr =  0.892
(70988, 656) Average  mse = 2.54810; corr = 0.892
CPU times: user 18.3 s, sys: 3.59 s, total: 21.9 s
Wall time: 12.3 s


Adjust params if required depending on cross validation results to tune model before retraining on full training data

## Retrain (given satisfactory cross validation)

In [13]:
# We retrain the model and then delete the training data, which is no longer needed
model, score_list, result_df = None, None, None # free the RAM occupied by the old model
gc.collect()
model = Ridge(copy_X=False) # we overwrite the training data
model.fit(X_train, y_train)

del X_train, y_train # free the RAM
gc.collect()

0

## Predict 

In [14]:
%%time
test_pred = model.predict(X_test)
del X_test
gc.collect()

CPU times: user 763 ms, sys: 300 ms, total: 1.06 s
Wall time: 363 ms


23

# Submission

The CITEseq test predictions have 48663 rows (i.e., cells) and 140 columns (i.e. proteins). 48663 * 140 = 6812820. The final submission will have 65744180 rows, of which the first 6812820 are for the CITEseq predictions and the remaining 58931360 for the Multiome predictions. 

In [17]:
%%time
if SUBMIT:
    # generate submission for CITEseq
    submission = pd.read_csv(FP_SUBMISSION,index_col='row_id', squeeze=True)
    submission.iloc[:len(test_pred.ravel())] = test_pred.ravel()
    assert not submission.isna().any()

    submission.to_csv('submission_citeseq_ridge.csv')
    display(submission)

row_id
0           0.293109
1           0.478517
2           0.836718
3           4.669548
4           5.313168
              ...   
65744175    0.000000
65744176    0.000000
65744177    0.000000
65744178    0.000000
65744179    0.000000
Name: target, Length: 65744180, dtype: float64

CPU times: user 2min 36s, sys: 4.35 s, total: 2min 40s
Wall time: 2min 48s
